In [3]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchcrf import CRF
import re
import time
import json
import numpy as np
from sklearn.metrics import classification_report
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
pd.set_option('max_colwidth',None)
from collections import Counter
from multiprocessing import Pool
import math
import gzip
import wordninja

d:\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 读取数据

In [150]:
df = pd.read_csv('./data/raw_data/已标注data.csv')

In [151]:
df = df.drop('MONEY	MONEY_NEW	DATE\tlabel'.split('\t'),axis=1)

In [152]:
def find_substring(long_str, short_str):
    indices = []
    length_long = len(long_str)
    length_short = len(short_str)
    i = 0
    while i <= length_long - length_short:
        if long_str[i:i+length_short] == short_str:
            indices.append([i, i + length_short - 1])
        i += 1
    return indices
text = '<NairaBag > Thank you, we received your payment for 3,000 and the remaining outstanding loan is 3,000.'
label = '3,000'
find_substring(text, label)

[[52, 56], [96, 100]]

In [153]:
label_list = []
too_many_amount = []
too_many_days = []
for i in tqdm(range(len(df))):
    label_dict = {}

    text = str(df['msg'].iloc[i])
    amount = str(df['amount'].iloc[i])
    days = str(df['days'].iloc[i])

    if amount != 'nan':
        loc = find_substring(text, amount)
        if len(loc) > 1:
            too_many_amount.append(1)
        else:
            label_dict['amount'] =amount
            too_many_amount.append(0)
    else:
        label_dict['amount'] = {}
        too_many_amount.append(0)
    if days != 'nan':
        loc = find_substring(text, days)
        if len(loc) > 1:
            too_many_days.append(1)
        else:
            label_dict['days'] = days
            too_many_days.append(0)
    else:
        label_dict['days'] = {}
        too_many_days.append(0)
    label_list.append(label_dict)
df['label'] = label_list
df['too_many_amount'] = too_many_amount
df['too_many_days'] = too_many_days

100%|██████████| 315808/315808 [00:09<00:00, 31938.80it/s]


In [154]:
df = df[(df['too_many_days']==0) & (df['too_many_amount']==0)]

In [155]:
df = df.drop(['label','too_many_amount','too_many_days'],axis=1)

In [156]:
class SplitJoinWords():
    def __init__(self, data_path):
        with open(data_path, 'r', encoding='utf-8') as f:
            self.corpus = f.read()
        self.dictionary = self.build_freq_stats()
        self.segmenter = wordninja.LanguageModel('conf/freq_stats.txt.gz')

    def words(self, text): 
        return re.findall('[a-zA-Z]+|[,.]', text)
    
    def build_freq_stats(self):
        freq_stats = Counter(self.words(self.corpus))
        sorted_keys = sorted(freq_stats, key=lambda k: freq_stats[k], reverse=True)
        with gzip.open('conf/freq_stats.txt.gz', 'wt', encoding='utf-8') as f:
            for word in sorted_keys:
                f.write(word+'\n')
        return set(sorted_keys)

    def split(self, word):
        return self.segmenter.split(word)

class Tokenizer():
    def __init__(self):
        self.pattern = re.compile(r'[\n\r\t]|(\d)|[\!\"\#\$\%\&\\\'\(\)\*\+\/\:\;\<\=\>\?\@\[\\\\\]\^\`\{\|\}\~\，\。\？\！\：\、\《\》\ ]|([\,\.\-\_])')
        
    def tokenize(self, sentence, segmenter):
        token_list = []
        tokens = re.split(self.pattern, sentence)
        for token in tokens:
            if token and token not in '\,\.\-\_':
                token_list.extend(segmenter.split(token))
            elif token and token in '\,\.\-\_':
                token_list.append(token)
        return token_list
segmenter = SplitJoinWords('data/raw_data/train.csv')
tokenizer = Tokenizer()

In [167]:
msg_tokens_list = []
amount_tokens_list = []
days_tokens_list = []
for i in tqdm(range(len(df))):
    msg = str(df['msg'].iloc[i])
    amount = str(df['amount'].iloc[i])
    days = str(df['days'].iloc[i])
    msg_tokens = tokenizer.tokenize(msg,segmenter)
    if amount != 'nan':
        amount_tokens = tokenizer.tokenize(df['amount'].iloc[i],segmenter)
    else:
        amount_tokens = None
    if days != 'nan':
        days_tokens = tokenizer.tokenize(df['days'].iloc[i],segmenter)
    else:
        days_tokens = None  
    msg_tokens_list.append(msg_tokens)
    amount_tokens_list.append(amount_tokens)
    days_tokens_list.append(days_tokens)

100%|██████████| 294771/294771 [02:18<00:00, 2132.38it/s]


In [171]:
df['msg_tokens'] = msg_tokens_list
df['amount'] = amount_tokens_list
df['days'] = days_tokens_list

In [176]:
def bieo(df, text_col, label_cols):
    bieo_col = []
    for i in tqdm(range(len(df))):
        msg_tokens = df[text_col].iloc[i]
        bieo_list = ['O'] * len(msg_tokens)
        for col in label_cols:
            label_toknes = df[col].iloc[i]
            for j in range(len(msg_tokens)):
                if label_toknes and msg_tokens[j:j+len(label_toknes)] == label_toknes:
                    bieo_list[j:j+len(label_toknes)-1] = ['I-' + col] * len(label_toknes)
                    bieo_list[j] = 'B-' + col
                    bieo_list[j+len(label_toknes)-1] = 'E-'  + col
        bieo_col.append(bieo_list)
    df['label'] = bieo_col
bieo(df, text_col = 'msg_tokens',label_cols = ['days', 'amount'])

100%|██████████| 294771/294771 [00:06<00:00, 45289.91it/s]


In [180]:
train, test = train_test_split(df, test_size=0.3)
val, test = train_test_split(test, test_size= 2/3)


In [183]:
with open('./data/raw_data/train.txt','w',encoding='utf-8') as f:
    for i in tqdm(range(len(train))):
        text_list = train['msg_tokens'].iloc[i]
        label_list = train['label'].iloc[i]
        for token, label in zip(text_list, label_list):
            f.write(token + '\t' + label + '\n')
        f.write('\n')

with open('./data/raw_data/val.txt','w',encoding='utf-8') as f:
    for i in tqdm(range(len(val))):
        text_list = val['msg_tokens'].iloc[i]
        label_list = val['label'].iloc[i]
        for token, label in zip(text_list, label_list):
            f.write(token + '\t' + label + '\n')
        f.write('\n')

with open('./data/raw_data/test.txt','w',encoding='utf-8') as f:
    for i in tqdm(range(len(test))):
        text_list = test['msg_tokens'].iloc[i]
        label_list = test['label'].iloc[i]
        for token, label in zip(text_list, label_list):
            f.write(token + '\t' + label + '\n')
        f.write('\n')

100%|██████████| 58955/58955 [00:01<00:00, 33686.35it/s]


# 数据清洗以及BIO标注

In [5]:
pd.set_option('max_colwidth',None)
def tokenizer(sentence):
    sentence = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', sentence)
    sentence = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', sentence)
    tokens = re.findall(r'\d|[^\w\s]|\w+', sentence)
    return tokens

def bio(text, label, label_class,tag_seq=None):
    text_split = tokenizer(text)
    if not tag_seq:
            tag_seq = ['O'] * len(text_split)
    label_split = tokenizer(label)
    start_indices = [i for i in range(len(text_split)) if text_split[i:i+len(label_split)] == label_split]
    if len(start_indices) == 0:
        return tag_seq
    
    for start_index in start_indices:
        end_index = start_index + len(label_split)

        if end_index - start_index == 1:
            tag_seq[start_index] = 'B-' + label_class
        else:
            tag_seq[start_index] = 'B-' + label_class
            tag_seq[end_index-1] = 'E-' + label_class
            for i in range(start_index+1, end_index-1):
                tag_seq[i] = 'I-' + label_class
    return tag_seq

def bio_df(df):
    df_ = df.copy()
    bio_col = []
    text_col = []
    for i, row in tqdm(df_.iterrows()):
        days = str(row['days'])
        amount = str(row['amount'])
        msg = str(row['msg'])
        token_seq = tokenizer(msg)
        label_seq = bio(msg, days, 'days')
        label_seq = bio(msg, amount, 'amount', label_seq)
        assert len(token_seq) == len(label_seq)
        bio_col.append(label_seq)
        text_col.append(token_seq)
    df_['BIO'] = bio_col
    df_['text'] = text_col
    train, test = train_test_split(df_,test_size=0.1)
    with open('comp_train.json', 'w') as f:
        for index, row in train.iterrows():
            # 将每行的text和label列的值转换为字典
            data = {'text': row['text'], 'label': row['BIO']}
            # 将字典转换为JSON字符串并写入文件
            f.write(json.dumps(data))
            f.write('\n')

    with open('comp_test.json', 'w') as f:
        for index, row in test.iterrows():
            # 将每行的text和label列的值转换为字典
            data = {'text': row['text'], 'label': row['BIO']}
            # 将字典转换为JSON字符串并写入文件
            f.write(json.dumps(data))
            f.write('\n')

In [6]:
bio_df(df)

315808it [00:36, 8716.46it/s] 


# 构建word2id和tag2id，id2word和id2tag

In [2]:
def build_word2id_tag2id(data_path):
    sentences = []
    labels = []
    with open(data_path,'r') as f:
            for line in f:
                sentence = json.loads(line)
                sentences.append(sentence['text'])
                labels.append(sentence['label'])
    word2id = {'<PAD>':0,'<UNK>':1}
    tag2id = {}
    for sentence, label in zip(sentences, labels):
        for word, tag in zip(sentence, label):
            if word not in word2id:
                word2id[word] = len(word2id)
            if tag not in  tag2id:
                tag2id[tag] = len(tag2id)
    return word2id, tag2id      
word2id, tag2id = build_word2id_tag2id(r'./comp_train.json')
id2word, id2tag = {v:k for k,v in word2id.items()}, {v:k for k,v in tag2id.items()}

# 封装数据

In [3]:
class MyDataset(Dataset):
    def __init__(self, data_path, word2id, tag2id):
        self.data_path = data_path
        self.word2id = word2id
        self.tag2id = tag2id
        self.texts = []
        self.labels = []
        # 加载数据
        with open(data_path,'r') as f:
            for line in f:
                sentence = json.loads(line)
                self.texts.append(sentence['text'])
                self.labels.append(sentence['label'])
        # mask
        self.mask = []
        for sentence in self.texts:
            mask = [1] * len(sentence)
            self.mask.append(mask)
        # 计算最大长度
        max_length = 0     
        for sentence_label in self.labels:
            length = len(sentence_label)
            if length > max_length:
                max_length = length

        # 填充句子
        for i in range(len(self.texts)):
            length = len(self.texts[i])
            if length < max_length:
                pad_length = max_length - length
                self.texts[i].extend(['<PAD>'] * pad_length)
                self.labels[i].extend(['O'] * pad_length)
                self.mask[i].extend([0] * pad_length)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sentence = self.texts[idx]
        label = self.labels[idx]
        mask = self.mask[idx]
        sentence_to_id = []
        label_to_id = []
        for word, tag, m in zip(sentence, label, mask):
            if word in self.word2id:
                sentence_to_id.append(self.word2id[word])
            else:
                sentence_to_id.append(self.word2id['<UNK>'])
            label_to_id.append(self.tag2id[tag])
        return torch.tensor(sentence_to_id), torch.tensor(label_to_id), torch.tensor(mask).bool()

# 数据封装与模型初始化

In [4]:
train = MyDataset('./comp_train.json',word2id, tag2id)
dev = MyDataset('./comp_test.json',word2id, tag2id)
train_dataloader = DataLoader(train, batch_size=128, shuffle=True)
dev_dataloader = DataLoader(dev, batch_size=512)

# 模型定义

In [7]:
# 构建基于bilstm+crf实现ner


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BilstmCrf(word2id, 
                    tag2id, 
                    embedding_dim=128, 
                    hidden_dim=200, 
                    num_layers=2,
                    dropout=0.3)
model.to(device)
# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# 训练

In [9]:
def train(model, model_name, train_dataloader, validate_dataloader, optimizer, scheduler, epochs):
    best_f1_score = 0
    early_stop = 0
    # 模型保存地址
    time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime(time.time()))
    model_saved_path = f"./{model_name}_{time_str}.pt"
    best_report = None
    for epoch in range(epochs):
        loss = train_single_epoch(model, train_dataloader, validate_dataloader, optimizer, epoch)
        scheduler.step()
        report = eval(model, validate_dataloader, id2tag)
        precision = report['macro avg']['precision']
        recall = report['macro avg']['recall']
        f1_score = report['macro avg']['f1-score']
        print(f'epoch:{epoch + 1} precision:{precision} recall:{recall} f1-score:{f1_score}' )
        if f1_score > best_f1_score:
            early_stop = 0
            best_f1_score = f1_score
            torch.save(model.state_dict(), model_saved_path)
            best_report = report
        else:
            early_stop += 1
            if early_stop == 3:
                print(pd.DataFrame(report))
                break
    print(pd.DataFrame(best_report))
    return

def train_single_epoch(model, train_dataloader, validate_dataloader, optimizer, epoch):
    epoch_loss = 0
    for i, (inputs, label, mask) in enumerate(train_dataloader):
        # 将数据移到GPU上
        model.train()
        inputs = inputs.to(device)
        label = label.to(device)
        mask = mask.to(device)
        # 清空梯度
        optimizer.zero_grad()

        # 前向传播计算损失
        loss = model.compute_loss(inputs, label, mask)

        # 反向传播计算梯度
        loss.backward()
        optimizer.step()

        # 统计损失
        epoch_loss += loss.item()
        # 打印每轮batch的loss
        if i == 0 or (i+1) % 10==0 or i==len(train_dataloader):
            print(f'epoch:{epoch + 1} batch {i+1}/{len(train_dataloader)}) Loss:{loss.item()}')
    return epoch_loss / len(train_dataloader)



def eval(model, validate_dataloader, id2tag):
    model.eval()

    true_labels = []
    predict_labels = []

    for (inputs, label, mask) in validate_dataloader:
        inputs = inputs.to(device)
        out = model.decode(inputs)
        for out_sentence, label_sentence in zip(out, label.tolist()):
            for predict_label, true_label in zip(out_sentence,label_sentence):
                true_labels.append(id2tag[true_label])
                predict_labels.append(id2tag[predict_label])
    report = classification_report(true_labels, predict_labels, output_dict=True)
    return report

In [10]:
train(model, 'Bilstm_CRF', train_dataloader,dev_dataloader, optimizer, epochs=50, scheduler=scheduler)

epoch:1 batch 1/2221) Loss:11880.912109375
epoch:1 batch 10/2221) Loss:12563.33984375
epoch:1 batch 20/2221) Loss:10768.126953125
epoch:1 batch 30/2221) Loss:10339.71484375
epoch:1 batch 40/2221) Loss:9217.990234375
epoch:1 batch 50/2221) Loss:8335.060546875
epoch:1 batch 60/2221) Loss:6162.5517578125
epoch:1 batch 70/2221) Loss:4114.32421875
epoch:1 batch 80/2221) Loss:3003.3466796875
epoch:1 batch 90/2221) Loss:2679.3505859375
epoch:1 batch 100/2221) Loss:2158.533447265625
epoch:1 batch 110/2221) Loss:2558.849609375
epoch:1 batch 120/2221) Loss:2190.23193359375
epoch:1 batch 130/2221) Loss:2252.865478515625
epoch:1 batch 140/2221) Loss:2034.83447265625
epoch:1 batch 150/2221) Loss:2036.118408203125
epoch:1 batch 160/2221) Loss:1852.30615234375
epoch:1 batch 170/2221) Loss:1905.473876953125
epoch:1 batch 180/2221) Loss:1625.839599609375
epoch:1 batch 190/2221) Loss:1729.02490234375
epoch:1 batch 200/2221) Loss:1623.231201171875
epoch:1 batch 210/2221) Loss:1761.4169921875
epoch:1 batc

d:\anaconda3\envs\pytorch\lib\site-packages\torchcrf\__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorCompare.cpp:413.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)
d:\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this

epoch:1 precision:0.8280896747094458 recall:0.7983012359942376 f1-score:0.8123559012925167
epoch:2 batch 1/2221) Loss:76.26997375488281
epoch:2 batch 10/2221) Loss:85.35252380371094
epoch:2 batch 20/2221) Loss:93.84513854980469
epoch:2 batch 30/2221) Loss:93.95509338378906
epoch:2 batch 40/2221) Loss:94.27284240722656
epoch:2 batch 50/2221) Loss:91.32664489746094
epoch:2 batch 60/2221) Loss:99.82341766357422
epoch:2 batch 70/2221) Loss:81.10442352294922
epoch:2 batch 80/2221) Loss:58.92112731933594
epoch:2 batch 90/2221) Loss:82.06031036376953
epoch:2 batch 100/2221) Loss:100.6032943725586
epoch:2 batch 110/2221) Loss:135.50535583496094
epoch:2 batch 120/2221) Loss:112.96754455566406
epoch:2 batch 130/2221) Loss:77.35137939453125
epoch:2 batch 140/2221) Loss:133.91244506835938
epoch:2 batch 150/2221) Loss:178.332763671875
epoch:2 batch 160/2221) Loss:111.77056121826172
epoch:2 batch 170/2221) Loss:180.7086181640625
epoch:2 batch 180/2221) Loss:119.13520812988281
epoch:2 batch 190/2221)

d:\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epoch:2 precision:0.8284561803130355 recall:0.8271853016225373 f1-score:0.8270912148981423
epoch:3 batch 1/2221) Loss:39.80754089355469
epoch:3 batch 10/2221) Loss:65.91925048828125
epoch:3 batch 20/2221) Loss:32.208770751953125
epoch:3 batch 30/2221) Loss:44.97972106933594
epoch:3 batch 40/2221) Loss:41.57355499267578
epoch:3 batch 50/2221) Loss:68.93804168701172
epoch:3 batch 60/2221) Loss:41.27845001220703
epoch:3 batch 70/2221) Loss:91.4185562133789
epoch:3 batch 80/2221) Loss:74.0404052734375
epoch:3 batch 90/2221) Loss:77.93690490722656
epoch:3 batch 100/2221) Loss:21.763397216796875
epoch:3 batch 110/2221) Loss:56.86851501464844
epoch:3 batch 120/2221) Loss:31.849945068359375
epoch:3 batch 130/2221) Loss:37.12626647949219
epoch:3 batch 140/2221) Loss:63.42279815673828
epoch:3 batch 150/2221) Loss:57.38053894042969
epoch:3 batch 160/2221) Loss:60.096954345703125
epoch:3 batch 170/2221) Loss:110.36955261230469
epoch:3 batch 180/2221) Loss:43.97471618652344
epoch:3 batch 190/2221) 

KeyboardInterrupt: 

# 模型预测与实体抽取

In [71]:
import re
from collections import defaultdict

def tokenizer(sentence):
    sentence = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', sentence)
    sentence = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', sentence)
    tokens = re.findall(r'\d|[^\w\s]|\w+', sentence)
    return tokens

def encode(inputs, word2id):
    sentence_to_id = [word2id[word] if word in word2id else word2id['<UNK>'] for word in inputs]
    return torch.tensor(sentence_to_id)

def single_predict(model_path, content, word2id, tag2id):
    id2word = {v:k for k,v in word2id.items()}
    id2tag = {v:k for k,v in tag2id.items()}
    model = BilstmCrf(word2id, tag2id, embedding_dim=128, hidden_dim=200, num_layers=2, dropout=0.3).to(device)
    model.load_state_dict(model_path)
    inputs = encode(tokenizer(content), word2id).unsqueeze(0)
    predict = model.decode(inputs)[0]
    entities = defaultdict(list)
    current_entity = []
    print(predict)
    for i, char_no in enumerate(predict):
        if inputs[0][i] == 0:
            break
        char_text = id2word[inputs.tolist()[0][i]]
        predict_tag_type = id2tag[char_no]
        if predict_tag_type.startswith('B'):
            current_entity = [char_text + '/' + predict_tag_type]
        elif predict_tag_type.startswith('I') or predict_tag_type.startswith('E'):
            if current_entity and current_entity[-1].split('/')[1][1:] == predict_tag_type[1:]:
                current_entity.append(char_text + '/' + predict_tag_type)
            else:
                current_entity = []
        elif predict_tag_type == 'O' and current_entity:
            entities[current_entity[-1].split('/')[1][2:]] = ''.join([w.split('/')[0] for w in current_entity])
            current_entity = []
    return entities


In [90]:
dic = torch.load('./Bilstm_CRF_20230521_232126.pt', map_location=lambda storage, loc: storage)
inputs = 'Your OK loan of N 36600.0 is due tomorrow Please make sure your bank card ending in 2837 has sufficient balance You can pay via USSD or bank transfer paycom o pay 5568815140'
entities = single_predict(dic, inputs, word2id, tag2id)
for key in entities.keys():
    print(f'{key}: {entities[key]}')

[0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 3, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
amount: 36600.0
days: tomorrow


In [79]:
tag2id

{'O': 0,
 'B-amount': 1,
 'I-amount': 2,
 'E-amount': 3,
 'B-days': 4,
 'I-days': 5,
 'E-days': 6}

In [ ]:
{"text": ["EaseCash", "reminds", "you", "that", "the", "1", "installment", "of", "your", "2", "9", "2", "1", "2", "NGN", "will", "be", "due", "in", "1", "?", "days", ".", "Maintaining", "a", "good", "credit", "standing", "will", "help", "you", "to", "increase", "your", "credit", "limit"], 
"label": ["O", "O", "O", "O", "O", "O", "O", "O", "O", "B-amount", "I-amount", "I-amount", "I-amount", "E-amount", "O", "O", "O", "O", "O", "B-days", "I-days", "E-days", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]}